In [1]:
import pandas as pd
import os

risk_free_path = os.path.join('data', "F-F_Research_Data_Factors_daily.CSV")
rf_df = pd.read_csv(risk_free_path)

In [2]:
rf_df['date'] = pd.to_datetime(rf_df['date'], format="%Y%m%d").dt.date

In [3]:
rf_df = rf_df.drop(rf_df.columns[1:4], axis=1)
rf_df.columns = ["date", "risk_free_return"]

In [4]:
rf_df.to_csv("data/risk_free_us.csv", index=0)

In [12]:
from auto_esn.esn.esn import FlexDeepESN
from auto_esn.esn.reservoir import activation as A
from auto_esn.esn.reservoir.initialization import CompositeInitializer, WeightInitializer
from auto_esn.esn.readout.nn_readout import AutoNNReadout
import torch
from sklearn.metrics import r2_score
import numpy as np 
import pandas as pd


device = torch.device('cpu')
dtype = torch.double
torch.set_default_dtype(dtype)
readout = A
class MyESN(FlexDeepESN):
    def __init__(self, readout = AutoNNReadout, input_size: int = 1, hidden_size: int = 500, output_dim: int = 1, bias: bool = False, initializer: WeightInitializer = WeightInitializer(), num_layers=2, activation=A.self_normalizing_default(), washout: int = 30):
        rd = readout(input_size*num_layers)
        super().__init__(rd, input_size, hidden_size, bias, initializer, num_layers, activation, washout)
        self.rd_class = readout
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.bias = bias
        self.initializer = initializer
        self.num_layers= num_layers
        self.activation = activation
        self.washout = washout

    def fit(self, X, y): 
        if  isinstance(X, pd.DataFrame):
            X = X.to_numpy()
            y = y.to_numpy()
        return super().fit(torch.from_numpy(X), torch.from_numpy(y))

    def predict(self, X):
        if  isinstance(X, pd.DataFrame):
            X = X.to_numpy()

        mapped_input = self.reservoir(torch.from_numpy(X))
        return self.readout(mapped_input)[:,-X.shape[0]]
        # return np.squeeze(self(torch.tensor(X, device=device)))
    
    def get_params(self, deep=False):
        return {"readout": self.rd_class, "input_size": self.input_size, "hidden_size": self.hidden_size,"output_dim": self.output_dim, "bias": self.bias,"initializer": self.initializer, "num_layers": self.num_layers,"activation": self.activation, "washout": self.washout}
    def set_params(self, **params):
        return MyESN(**params)

from experiment import run_experiment, build_data




data = build_data(
    stock='GOOG',
    market='^spx',
    risk_free='risk_free_us.csv'
).fillna(0)

i = CompositeInitializer()\
    .with_seed(23)\
    # .sparse(density=1)\
    # .spectral_noisy(spectral_radius=1.5, noise_magnitude=0.1)\
    # .uniform()\
    # .regular_graph(4)\
    # .scale(0.9)\
    
w = WeightInitializer()
w.weight_hh_init = i

grid = {
    'input_size': [21],
    'hidden_size': [21],
    'output_dim': [1],
    'bias': [False],
    'initializer': [w],
    'num_layers': [2],
    'activation': [A.self_normalizing_default()],
    'washout': [0]
}



model = MyESN(input_size=21, initializer=w)

run_experiment(
    estimator=model,
    data=data,
    param_dist=grid,
    input_columns=['beta'],
    lag_size=21,
    split_size=5,
    opt_samples=1,
    # scoring=r2_score
)

c:\Users\piotr\Workspaces\estimating-beta\experiment.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'beta'] = eval_beta(data, beta_corr_window)
59it [00:01, 50.83it/s]


ValueError: 
All the 59 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\piotr\Workspaces\estimating-beta\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\piotr\AppData\Local\Temp\ipykernel_7892\2658662950.py", line 33, in fit
    return super().fit(torch.from_numpy(X), torch.from_numpy(y))
  File "c:\Users\piotr\Workspaces\estimating-beta\.venv\lib\site-packages\auto_esn\esn\esn.py", line 21, in fit
    self.readout.fit(mapped_input, target, washout=self.washout)
  File "c:\Users\piotr\Workspaces\estimating-beta\.venv\lib\site-packages\auto_esn\esn\readout\nn_readout.py", line 60, in f